<a href="https://colab.research.google.com/github/Angeal13/ClassifierML/blob/master/IrisFlowerClassificator_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import,division,print_function,unicode_literals

import tensorflow as tf

import pandas as pd

In [0]:
CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginia'] #Defince constant that will help later

In [0]:
train_path=tf.keras.utils.get_file(
    "iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path=tf.keras.utils.get_file(
    "iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test  = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)

In [0]:
train.head()

In [0]:
train_y = train.pop('Species')
test_y  = test.pop('Species')

In [0]:
def input_fn(features,labels,training=True,batch_size=256):
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels)) #Covert the inputs into a DataSest

  if training: #Shuffle and repeat if you are in training mode
    dataset=dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)

In [0]:
#Feature columns describe how to use the input
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

In [0]:
#Build a DNN with n-hidden layers with a certain number of nodes to choose with is a certain number of classes
classifier=tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30,10],
    # The model must choose between 3 classes.
    n_classes=3)

In [0]:
#Training the model unsig LAMBDA fuction
#We use LAMBDA becuse we do not have a function that would retunr the input_fn object to us
#Usig Steps instead of Epoch, to only look at the number of objects desire, both can be used for proceding adn processing
classifier.train(
    input_fn=lambda:input_fn(train,train_y,training=True), steps=5000)

In [0]:
#Evaluate the model Using LAMBDA as explain in the previous set
eval_result=classifier.evaluate(
    input_fn=lambda:input_fn(test,test_y,training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [0]:
def input_fn(features,batch_size=256):
  #Convert the inputs to a DataSet without Labels.
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}

print("Please type numeric values as prompted")
for feature in features:
  valid=True
  while valid:
    val=input(feature + ": ")
    if not val.isdigit():valid =False
  
  predict[feature]=[float(val)]

predictions=classifier.predict(input_fn=lambda:input_fn(predict))
for pred_dict in predictions:
  class_id=pred_dict['class_ids'][0]
  probability= pred_dict['probabilities'][class_id]

  print('Predictions is "{}" ({:.1f}%)'.format(
      SPECIES[class_id],100 * probability))